In [ ]:
# 2018A 1)拟合参数

In [1]:
import numpy as np

In [2]:
# 附件1. 专用服装材料的参数值				
# 分层	"密度
# (kg/m）"	"比热
# (J/(kg·ºC))"	"热传导率
# (W/(m·ºC))"	"厚度
# (mm)"
# I层	300	1,377	0.08	0.60
# II层	862	2,100	0.37	0.6-25
# III层	74.20	1,726	0.05	3.60
# IV层	1.18	1,005	0.03	0.6-6.4

In [3]:
rho = np.array([300, 862, 74.20, 1.18])
c = np.array([1377, 2100, 1726, 1005])
k = np.array([0.08, 0.37, 0.05, 0.03])
t = np.array([0.6e-3, 6e-3, 3.6e-3, 5e-3])

In [4]:
dx = 0.02e-3
dt = 0.1

In [5]:
b_ = [0]
for each_t in t:
    b_.append(b_[-1] + int(each_t / dx))
b_

[0, 29, 329, 508, 757]

In [6]:
T = 5400
n_ = int(T // dt)
n_

53999

In [7]:
M_B = np.zeros((b_[-1] + 1, b_[-1] + 1))
dt_over_dx_squared = dt / (dx ** 2)
for part in range(1, len(b_)):
    ld = dt_over_dx_squared * k[part - 1] / (c[part - 1] * rho[part - 1])
    for x_ in range(b_[part - 1], b_[part]):
        M_B[x_][x_] = 1 + 2 * ld
        if x_ != 0:
            M_B[x_][x_ - 1] = -ld
        if x_ != b_[-1]:
            M_B[x_][x_ + 1] = -ld
for part, border in enumerate(b_[1:-1]):
    nu_last = k[part]
    nu_next = k[part + 1]
    M_B[border][border] = nu_last + nu_next
    M_B[border][border - 1] = -nu_last
    M_B[border][border + 1] = -nu_next
M_B[0][1] = -1
M_B[-1][-2] = -1
for each in M_B[28:31].tolist():
    print(each[20:40])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -48.41442749939481, 97.82885499878962, -48.41442749939481, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08, 0.45, -0.37, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -51.09932604132139, 103.19865208264278, -51.09932604132139, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
T_env = 70
T_body = 37
u_0 = None
u_last = None

def set_convection_param(h0, h1):
    mu0 = dx * h0 / k[0]
    mu1 = dx * h1 / k[-1]
    M_B[0][0] = 1 + mu0
    M_B[-1][-1] = 1 + mu1
    global u_0, u_last
    u_0 = mu0 * T_env
    u_last = mu1 * T_body

set_convection_param(118.5, 8.37)

In [9]:
M_B

array([[ 1.02962500e+00, -1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.84144275e+01,  9.78288550e+01, -4.84144275e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.84144275e+01,  9.78288550e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.26496213e+04, -6.32431065e+03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.32431065e+03,  1.26496213e+04, -6.32431065e+03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -1.00000000e+00,  1.00558000e+00]])

In [10]:
T = np.ndarray((n_, b_[-1] + 1))
T.fill(T_body)

In [11]:
u_0

2.07375

In [12]:
mask = np.zeros_like(T[0], dtype=bool)
mask[b_[1:-1]] = True

M_B_inv = np.linalg.inv(M_B)

for t_ in range(1, n_):
    if t_ % int(n_ / 20) == 0:
        print(t_ / n_)
    equ_b = T[t_ - 1].copy()
    equ_b[0] = u_0
    equ_b[-1] = u_last
    equ_b[mask] = 0
    T[t_] = M_B_inv @ equ_b
    # T[t_] = np.linalg.solve(M_B, equ_b)

0.049982407081612626
0.09996481416322525
0.14994722124483786
0.1999296283264505
0.24991203540806312


0.2998944424896757
0.34987684957128834
0.399859256652901
0.4498416637345136
0.49982407081612623
0.5498064778977388
0.5997888849793515
0.6497712920609641
0.6997536991425767
0.7497361062241893
0.799718513305802
0.8497009203874146
0.8996833274690272
0.9496657345506399
0.9996481416322525


In [13]:
T[-1]

array([69.2840725 , 69.26286315, 69.24165379, 69.22044444, 69.19923509,
       69.17802574, 69.15681639, 69.13560703, 69.11439768, 69.09318833,
       69.07197898, 69.05076962, 69.02956027, 69.00835092, 68.98714157,
       68.96593221, 68.94472286, 68.92351351, 68.90230416, 68.88109481,
       68.85988545, 68.8386761 , 68.81746675, 68.7962574 , 68.77504804,
       68.75383869, 68.73262934, 68.71141999, 68.69021064, 68.66900128,
       68.66441548, 68.65982967, 68.65524387, 68.65065806, 68.64607225,
       68.64148645, 68.63690064, 68.63231484, 68.62772903, 68.62314322,
       68.61855742, 68.61397161, 68.60938581, 68.6048    , 68.6002142 ,
       68.59562839, 68.59104258, 68.58645678, 68.58187097, 68.57728517,
       68.57269936, 68.56811355, 68.56352775, 68.55894194, 68.55435614,
       68.54977033, 68.54518452, 68.54059872, 68.53601291, 68.53142711,
       68.5268413 , 68.5222555 , 68.51766969, 68.51308388, 68.50849808,
       68.50391227, 68.49932647, 68.49474066, 68.49015485, 68.48

In [17]:
import plotly.graph_objects as go

# Assuming T is your numpy array with temperature data
x, y = T.shape
x_interval = 40
y_interval = 40
x_values = list(range(0, x, x_interval))
y_values = list(range(0, y, y_interval))
X, Y = np.meshgrid(y_values, x_values)

fig = go.Figure(data=[go.Surface(z=T[::x_interval, ::y_interval], x=X, y=Y)])
fig.show()